In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils
import tensorflow as tf
import numpy as np
import h5py

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
# To get the images and labels from file
with h5py.File('Galaxy10_DECals.h5', 'r') as F:
    images = np.array(F['images'])
    labels = np.array(F['ans'])

# To convert the labels to categorical 10 classes
labels = utils.to_categorical(labels, 10)

# To convert to desirable type
labels = labels.astype(np.float32)
images = images.astype(np.float32)

In [13]:
from sklearn.model_selection import train_test_split

# train test split
train_idx, test_idx = train_test_split(np.arange(labels.shape[0]), test_size=0.1)
train_images, train_labels, test_images, test_labels = images[train_idx], labels[train_idx], images[test_idx], labels[test_idx]

# train validation split
train_idx, val_idx = train_test_split(np.arange(train_images.shape[0]), test_size=0.2)
train_images, train_labels, val_images, val_labels = train_images[train_idx], train_labels[train_idx], \
                                                     train_images[val_idx], train_labels[val_idx]

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1000

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [6]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [11]:
img_shape = images.shape[1:]
inputs = keras.Input(shape=img_shape)
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="base_galaxy_class_w_aug.keras",
        save_best_only=True,
        monitor="val_loss")
]

In [15]:
history = model.fit(
        train_images, train_labels,
        epochs=100,
        batch_size=512,
        validation_data=(val_images, val_labels),
        callbacks=callbacks)

2023-08-19 23:11:12.327212: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 10041950208 exceeds 10% of free system memory.


Epoch 1/100
25/25 [==============================] - 12s 468ms/step - loss: 2.3097 - accuracy: 0.1478 - val_loss: 2.2400 - val_accuracy: 0.1434
Epoch 2/100
25/25 [==============================] - 8s 315ms/step - loss: 2.1850 - accuracy: 0.1929 - val_loss: 6.7737 - val_accuracy: 0.1434
Epoch 3/100
25/25 [==============================] - 7s 283ms/step - loss: 2.2261 - accuracy: 0.2192 - val_loss: 1.9491 - val_accuracy: 0.2794
Epoch 4/100
25/25 [==============================] - 7s 290ms/step - loss: 1.9152 - accuracy: 0.2808 - val_loss: 1.8155 - val_accuracy: 0.3050
Epoch 5/100
25/25 [==============================] - 8s 333ms/step - loss: 1.8086 - accuracy: 0.3250 - val_loss: 1.8201 - val_accuracy: 0.3069
Epoch 6/100
25/25 [==============================] - 7s 298ms/step - loss: 1.7016 - accuracy: 0.3706 - val_loss: 1.6512 - val_accuracy: 0.3890
Epoch 7/100
 1/25 [>.............................] - ETA: 6s - loss: 1.5713 - accuracy: 0.4492

2023-08-19 23:12:17.192779: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-19 23:12:17.255544: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


25/25 [==============================] - 8s 328ms/step - loss: 1.6002 - accuracy: 0.4035 - val_loss: 1.6136 - val_accuracy: 0.4078
Epoch 8/100
25/25 [==============================] - 7s 291ms/step - loss: 1.5310 - accuracy: 0.4361 - val_loss: 1.5473 - val_accuracy: 0.4259
Epoch 9/100
25/25 [==============================] - 7s 297ms/step - loss: 1.4231 - accuracy: 0.4835 - val_loss: 1.5878 - val_accuracy: 0.4184
Epoch 10/100
25/25 [==============================] - 8s 318ms/step - loss: 1.3692 - accuracy: 0.4959 - val_loss: 1.5268 - val_accuracy: 0.4400
Epoch 11/100
 1/25 [>.............................] - ETA: 6s - loss: 1.4168 - accuracy: 0.4902

2023-08-19 23:12:47.930307: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-19 23:12:47.993719: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


...

25/25 [==============================] - 7s 292ms/step - loss: 0.7000 - accuracy: 0.7484 - val_loss: 1.4483 - val_accuracy: 0.5211
Epoch 22/100
25/25 [==============================] - 8s 333ms/step - loss: 0.6429 - accuracy: 0.7777 - val_loss: 1.3471 - val_accuracy: 0.5894
Epoch 23/100
25/25 [==============================] - 7s 301ms/step - loss: 0.5848 - accuracy: 0.7981 - val_loss: 1.6123 - val_accuracy: 0.5296
Epoch 24/100
25/25 [==============================] - 8s 336ms/step - loss: 0.5654 - accuracy: 0.8012 - val_loss: 1.5412 - val_accuracy: 0.5753
Epoch 25/100
 1/25 [>.............................] - ETA: 6s - loss: 0.5046 - accuracy: 0.8223

2023-08-19 23:14:37.476258: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-19 23:14:37.536772: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 2/25 [=>............................] - ETA: 6s - loss: 0.4933 - accuracy: 0.8242

2023-08-19 23:14:37.738545: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-19 23:14:37.801340: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


25/25 [==============================] - 7s 299ms/step - loss: 0.5314 - accuracy: 0.8154 - val_loss: 1.5105 - val_accuracy: 0.5781
Epoch 26/100
25/25 [==============================] - 8s 336ms/step - loss: 0.4967 - accuracy: 0.8243 - val_loss: 1.5218 - val_accuracy: 0.5819
Epoch 27/100
25/25 [==============================] - 8s 308ms/step - loss: 0.4424 - accuracy: 0.8481 - val_loss: 1.6044 - val_accuracy: 0.5653
Epoch 28/100
25/25 [==============================] - 8s 335ms/step - loss: 0.4120 - accuracy: 0.8579 - val_loss: 1.8456 - val_accuracy: 0.5224
Epoch 29/100
25/25 [==============================] - 8s 306ms/step - loss: 0.3834 - accuracy: 0.8655 - val_loss: 1.6128 - val_accuracy: 0.5700
Epoch 30/100
25/25 [==============================] - 8s 328ms/step - loss: 0.3758 - accuracy: 0.8677 - val_loss: 1.8483 - val_accuracy: 0.5738
Epoch 31/100
25/25 [==============================] - 7s 297ms/step - loss: 0.3273 - accuracy: 0.8861 - val_loss: 1.6900 - val_accuracy: 0.5509
Epoch

In [17]:
test_model = keras.models.load_model("base_galaxy_class_w_aug.keras")
test_loss, test_acc = test_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

56/56 [==============================] - 1s 10ms/step - loss: 1.3676 - accuracy: 0.5366
Test accuracy: 0.537


## Trying Pretrained Models

In [12]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(256, 256, 3))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [14]:
def get_feat(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels = get_feat(train_dataset)
val_features, val_labels = get_feat(val_dataset)
test_features, test_labels = get_feat(test_dataset)

8/8 [==============================] - 0s 67ms/step


In [15]:
train_features.shape

(12769, 8, 8, 512)

In [16]:
inputs = keras.Input(shape=(8, 8, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_tla = keras.Model(inputs, outputs)

In [56]:
model_tla.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="feature_extraction.keras",
save_best_only=True,
monitor="val_loss")
]
history = model_tla.fit(
train_features, train_labels,
epochs=20,
validation_data=(val_features, val_labels),
callbacks=callbacks)

Epoch 1/20
400/400 [==============================] - 2s 5ms/step - loss: 93.9866 - accuracy: 0.4309 - val_loss: 37.8218 - val_accuracy: 0.4945
Epoch 2/20
400/400 [==============================] - 2s 4ms/step - loss: 32.5169 - accuracy: 0.5744 - val_loss: 26.5428 - val_accuracy: 0.4372
Epoch 3/20
400/400 [==============================] - 2s 4ms/step - loss: 16.4280 - accuracy: 0.6106 - val_loss: 30.1123 - val_accuracy: 0.4056
Epoch 4/20
400/400 [==============================] - 2s 4ms/step - loss: 10.9547 - accuracy: 0.6339 - val_loss: 15.4315 - val_accuracy: 0.4820
Epoch 5/20
400/400 [==============================] - 2s 4ms/step - loss: 8.6071 - accuracy: 0.6610 - val_loss: 16.2910 - val_accuracy: 0.4500
Epoch 6/20
400/400 [==============================] - 2s 4ms/step - loss: 7.7222 - accuracy: 0.6853 - val_loss: 19.6466 - val_accuracy: 0.4416
Epoch 7/20
400/400 [==============================] - 2s 4ms/step - loss: 7.2332 - accuracy: 0.7064 - val_loss: 17.3680 - val_accuracy: 0.

In [17]:
test_model = keras.models.load_model("feature_extraction.keras")
test_loss, test_acc = test_model.evaluate(test_features, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

56/56 [==============================] - 0s 2ms/step - loss: 7.9115 - accuracy: 0.6669
Test accuracy: 0.667


### Feature Extraction with Data Augmentation

In [34]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [31]:
keras.mixed_precision.set_global_policy("mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100-SXM4-40GB, compute capability 8.0


In [36]:
model_tla = keras.models.load_model("feature_extraction.keras")

In [ ]:
top_layers = model_tla.layers[1:]
top_layers = keras.Sequential(top_layers)

In [43]:
inputs = keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
outputs = top_layers(x)
model_tlb = keras.Model(inputs=inputs, outputs=outputs)

model_tlb.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 tf.__operators__.getitem_8   (None, 256, 256, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_8 (TFOpLambd  (None, 256, 256, 3)      0         
 a)                                                              
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 sequential_1 (Sequential)   (None, 10)                8391

In [48]:

model_tlb.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_ext_w_aug.keras",
        save_best_only=True,
        monitor="val_loss")
]

history = model_tlb.fit(
    train_images, train_labels,
    epochs=30,
    batch_size=128,
    validation_data=(val_images, val_labels),
    callbacks=callbacks)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

## Building a mini Xception-like Model

In [10]:
inputs = keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)

x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

# initializing pyramid architecture
for size in [32, 64, 128, 256, 512]:
    residual = x
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)
    
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    
    residual = layers.Conv2D(
            size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_x = keras.Model(inputs=inputs, outputs=outputs)

In [11]:
model_x.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 256, 256, 3)  0           ['input_4[0][0]']                
                                                                                                  
 rescaling_3 (Rescaling)        (None, 256, 256, 3)  0           ['sequential[3][0]']             
                                                                                                  
 conv2d_4 (Conv2D)              (None, 252, 252, 32  2400        ['rescaling_3[0][0]']        

In [19]:
model_x.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="mini_xception.keras",
        save_best_only=True,
        monitor="val_loss"),
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10,
)
]

history = model_x.fit(
    train_dataset,
    epochs=100,
    batch_size=128,
    validation_data=val_dataset,
    callbacks=callbacks)

Epoch 1/100
100/100 [==============================] - 106s 1s/step - loss: 1.5273 - accuracy: 0.4488 - val_loss: 2.3955 - val_accuracy: 0.1171
Epoch 2/100
100/100 [==============================] - 104s 1s/step - loss: 1.2666 - accuracy: 0.5548 - val_loss: 2.4426 - val_accuracy: 0.1688
Epoch 3/100
100/100 [==============================] - 104s 1s/step - loss: 1.1160 - accuracy: 0.6174 - val_loss: 2.2883 - val_accuracy: 0.1591
Epoch 4/100
100/100 [==============================] - 103s 1s/step - loss: 1.0079 - accuracy: 0.6610 - val_loss: 1.8278 - val_accuracy: 0.3442
Epoch 5/100
100/100 [==============================] - 103s 1s/step - loss: 0.9344 - accuracy: 0.6931 - val_loss: 1.2075 - val_accuracy: 0.5888
Epoch 6/100
100/100 [==============================] - 103s 1s/step - loss: 0.8548 - accuracy: 0.7150 - val_loss: 1.3800 - val_accuracy: 0.4933
Epoch 7/100
100/100 [==============================] - 103s 1s/step - loss: 0.8203 - accuracy: 0.7208 - val_loss: 1.4432 - val_accuracy:

In [21]:
test_model = keras.models.load_model("mini_xception.keras")
test_loss, test_acc = test_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

56/56 [==============================] - 1s 15ms/step - loss: 0.8035 - accuracy: 0.7356
Test accuracy: 0.736
